# Preparing Dataset

Load Dataset

In [ ]:
file_path = "dataset.txt"
with open(file_path, "r") as file:
    data = file.read().splitlines()

data[:5]

['User;Response',
 'Apa itu rendang?;Rendang adalah masakan daging berasal dari Minangkabau yang dimasak lama dengan santan dan rempah hingga kering.',
 'Apa bahan utama membuat rendang sapi?;Bahan utamanya adalah daging sapi, santan kelapa tua, dan campuran bumbu halus serta rempah daun.',
 'Bagian daging sapi apa yang terbaik untuk rendang?;Paha belakang (knuckle) adalah yang terbaik karena teksturnya padat dan tidak mudah hancur.',
 'Mengapa rendang dimasak sangat lama?;Tujuannya agar santan terkaramelisasi menjadi minyak dan bumbu meresap sempurna ke dalam serat daging.']

Convert ke format JSON

In [2]:
import json

def txt_qa_to_jsonl(input_path: str, output_path: str, encoding: str = "utf-8"):

    with open(input_path, "r", encoding=encoding) as f:
        lines = f.read().splitlines()

    lines = lines[1:]

    samples = []

    for idx, line in enumerate(lines):
        if ";" not in line:
            continue

        instruction, response = line.split(";", 1)

        instruction = instruction.strip()
        response = response.strip()

        if not instruction or not response:
            continue

        text = (
            "### Instruction:\n"
            f"{instruction}\n\n"
            "### Response:\n"
            f"{response}"
        )

        samples.append({"text": text})

    with open(output_path, "w", encoding=encoding) as f:
        for sample in samples:
            f.write(json.dumps(sample, ensure_ascii=False) + "\n")


In [ ]:
txt_qa_to_jsonl("dataset_850.txt", "dataset.jsonl")

# Load Model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt2-small-indonesian")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("flax-community/gpt2-small-indonesian")
model.to(device)

c:\Users\THINKPAD\anaconda3\envs\rendang_gpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Tokenization

In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="dataset.jsonl", split="train")

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=256
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Generating train split: 869 examples [00:00, 76097.67 examples/s]
Map: 100%|██████████| 869/869 [00:00<00:00, 2947.88 examples/s]


In [6]:
dataset[:5]

{'text': ['### Instruction:\nApa itu rendang?\n\n### Response:\nRendang adalah masakan daging berasal dari Minangkabau yang dimasak lama dengan santan dan rempah hingga kering.',
  '### Instruction:\nApa bahan utama membuat rendang sapi?\n\n### Response:\nBahan utamanya adalah daging sapi, santan kelapa tua, dan campuran bumbu halus serta rempah daun.',
  '### Instruction:\nBagian daging sapi apa yang terbaik untuk rendang?\n\n### Response:\nPaha belakang (knuckle) adalah yang terbaik karena teksturnya padat dan tidak mudah hancur.',
  '### Instruction:\nMengapa rendang dimasak sangat lama?\n\n### Response:\nTujuannya agar santan terkaramelisasi menjadi minyak dan bumbu meresap sempurna ke dalam serat daging.',
  '### Instruction:\nApa perbedaan gulai, kalio, dan rendang?\n\n### Response:\nGulai masih berkuah encer, kalio berkuah kental berminyak, dan rendang sudah kering serta berwarna gelap.']}

# Setup LoRA

In [7]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn"] 
)


model = get_peft_model(model, peft_config)
model.print_trainable_parameters() 

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


c:\Users\THINKPAD\anaconda3\envs\rendang_gpt\Lib\site-packages\peft\tuners\lora\layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


# Training

In [8]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-rendang-850",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=10,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    use_cpu=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

print("Mulai training...")
trainer.train()

model.save_pretrained("./gpt2-rendang-final-850")
print("Selesai! Model tersimpan di folder 'gpt2-rendang-final-850'")

Mulai training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.162600
20,4.021900
30,3.748400
40,3.418200
50,2.976500
60,2.548700
70,2.493500
80,2.277500
90,2.219700
100,2.226300


Selesai! Model tersimpan di folder 'gpt2-rendang-final-850'


# Eval

In [9]:
import torch
import math
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_path = "flax-community/gpt2-small-indonesian"
adapter_path = "./gpt2-rendang-final"

print("Sedang memuat model hasil training...")
model_eval = AutoModelForCausalLM.from_pretrained(base_model_path)
model_eval = PeftModel.from_pretrained(model_eval, adapter_path)

model_eval.to(device)
model_eval.eval()

tokenizer_eval = AutoTokenizer.from_pretrained(base_model_path)
tokenizer_eval.pad_token = tokenizer_eval.eos_token

print("✅ Model berhasil dimuat dengan adapter LoRA")

def calculate_perplexity_and_loss(text_list, model, tokenizer):
    encodings = tokenizer("\n\n".join(text_list), return_tensors="pt")
    max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    total_loss = 0
    total_batches = 0
    prev_end_loc = 0
    
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - begin_loc  # Perbaikan: hitung dari begin_loc
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len
            
            # Simpan loss
            total_loss += outputs.loss.item()
            total_batches += 1

        nlls.append(neg_log_likelihood)
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    avg_loss = total_loss / total_batches if total_batches > 0 else 0
    
    return ppl.item(), avg_loss

texts_to_eval = dataset["text"][:50]

print("Sedang menghitung Perplexity dan Loss...")
ppl_score, loss_score = calculate_perplexity_and_loss(texts_to_eval, model_eval, tokenizer_eval)

print(f"\n{'='*50}")
print(f"📊 HASIL EVALUASI MODEL")
print(f"{'='*50}")
print(f"✅ Perplexity Score: {ppl_score:.4f}")
print(f"✅ Average Loss: {loss_score:.4f}")
print(f"{'='*50}")
print("Catatan:")
print("- Semakin rendah Perplexity (mendekati 1), semakin baik model")
print("- Semakin rendah Loss, semakin baik akurasi prediksi")
print(f"{'='*50}\n")

Sedang memuat model hasil training...
✅ Model berhasil dimuat dengan adapter LoRA
Sedang menghitung Perplexity dan Loss...

📊 HASIL EVALUASI MODEL
✅ Perplexity Score: 76.4348
✅ Average Loss: 2.8407
Catatan:
- Semakin rendah Perplexity (mendekati 1), semakin baik model
- Semakin rendah Loss, semakin baik akurasi prediksi



In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk

# Download NLTK data jika belum ada
nltk.download('punkt')

base_model_path = "flax-community/gpt2-small-indonesian"
adapter_path = "./gpt2-rendang-final"

print("Sedang memuat model hasil training...")
model_eval = AutoModelForCausalLM.from_pretrained(base_model_path)
model_eval = PeftModel.from_pretrained(model_eval, adapter_path)

model_eval.to(device)
model_eval.eval()

tokenizer_eval = AutoTokenizer.from_pretrained(base_model_path)
tokenizer_eval.pad_token = tokenizer_eval.eos_token

print("✅ Model berhasil dimuat dengan adapter LoRA")

def calculate_bleu_score(text_list, model, tokenizer):
    """
    Hitung BLEU score dengan membandingkan teks asli sebagai referensi
    dan teks yang di-generate sebagai candidate
    """
    bleu_scores = []
    smoothing_function = SmoothingFunction().method1
    
    for text in text_list:
        # Split teks menjadi instruction dan response
        if "### Response:" in text:
            instruction_part = text.split("### Instruction:\n")[1].split("\n\n### Response:")[0]
            reference_response = text.split("### Response:\n")[1]
        else:
            continue
        
        # Generate response dari instruction
        prompt = f"### Instruction:\n{instruction_part}\n\n### Response:\n"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False,  # Gunakan greedy decoding untuk konsistensi
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Ambil bagian response
        if "### Response:" in generated_text:
            generated_response = generated_text.split("### Response:\n")[1]
        else:
            generated_response = generated_text
        
        # Tokenize untuk BLEU calculation
        reference_tokens = reference_response.lower().split()
        generated_tokens = generated_response.lower().split()
        
        # Hitung BLEU score (menggunakan unigram dan bigram)
        bleu = sentence_bleu(
            [reference_tokens],
            generated_tokens,
            weights=(0.5, 0.5),  # unigram 50%, bigram 50%
            smoothing_function=smoothing_function
        )
        bleu_scores.append(bleu)
    
    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    return avg_bleu

texts_to_eval = dataset["text"][:50]

print("Sedang menghitung BLEU Score...")
bleu_score = calculate_bleu_score(texts_to_eval, model_eval, tokenizer_eval)

print(f"\n{'='*50}")
print(f"📊 HASIL EVALUASI MODEL")
print(f"{'='*50}")
print(f"✅ BLEU Score: {bleu_score:.4f}")
print(f"{'='*50}")
print("Catatan:")
print("- BLEU Score berkisar 0-1 (semakin tinggi semakin baik)")
print("- BLEU Score mengukur kesamaan output dengan referensi")
print(f"{'='*50}\n")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\THINKPAD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sedang memuat model hasil training...
✅ Model berhasil dimuat dengan adapter LoRA
Sedang menghitung BLEU Score...

📊 HASIL EVALUASI MODEL
✅ BLEU Score: 0.0570
Catatan:
- BLEU Score berkisar 0-1 (semakin tinggi semakin baik)
- BLEU Score mengukur kesamaan output dengan referensi



# Test

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. Setup Model (Sama seperti evaluasi)
base_model_path = "flax-community/gpt2-small-indonesian"
adapter_path = "./gpt2-rendang-final"
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Memuat model untuk tes...")
model = AutoModelForCausalLM.from_pretrained(base_model_path)
model = PeftModel.from_pretrained(model, adapter_path)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token

def generate_resep(pertanyaan):
    # Format prompt HARUS SAMA PERSIS dengan saat training
    prompt = f"### Instruction:\n{pertanyaan}\n\n### Response:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate jawaban
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,      # Batasi panjang jawaban biar gak ngelantur
            do_sample=True,          # Supaya jawaban bervariasi
            temperature=0.4,         # Kreativitas (0.1 kaku, 1.0 liar)
            top_k=50,                # Ambil 50 kata terbaik
            top_p=0.95,              # Ambil probabilitas kumulatif 95%
            repetition_penalty=1.2,  # Cegah pengulangan kata
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode hasil (ubah angka jadi teks)
    hasil_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Ambil bagian Response saja
    if "### Response:" in hasil_text:
        jawaban = hasil_text.split("### Response:\n")[1]
    else:
        jawaban = hasil_text
        
    return jawaban.strip()

Memuat model untuk tes...


In [6]:
print("-" * 30)
pertanyaan_kamu = "Bagaimana cara membuat rendang?"
print(f"Pertanyaan: {pertanyaan_kamu}")
print("Model sedang berpikir...")
print("-" * 30)

jawaban_model = generate_resep(pertanyaan_kamu)
print(f"Jawaban Model:\n{jawaban_model}")
print("-" * 30)

------------------------------
Pertanyaan: Bagaimana cara membuat rendang?
Model sedang berpikir...
------------------------------
Jawaban Model:
Cara membuat rendang adalah dengan menggunakan daging sapi yang sudah dipotong-potong dan kemudian diulek hingga halus. Setelah itu, baru kemudian dicampur dengan bumbu rempah seperti lengkuas dan daun salam.
------------------------------


# Analisis

### Keamanan 
Sangat aman karena semua proses pengumpulan data, training, dan inference dilakukan di lokal dan tidak dikirim ke API publik (third party)

### Privasi Pengguna
Perlindungan privasi pengguna dari sisi aplikasi sudah memadai karena menggunakan sistem sesi sementara, di mana riwayat percakapan tidak disimpan permanen dan akan langsung hilang begitu pengguna menutup browser. Namun, terdapat catatan kritis pada sisi model (Model 2.0) yang terbukti mengalami Data Leakage dengan memunculkan teks "Baca juga artikel...", menandakan bahwa model dapat memuntahkan data mentah dari masa lalunya. Ini menjadi peringatan keras bahwa dataset pelatihan tidak boleh mengandung informasi identitas pribadi (PII) karena berisiko muncul kembali dalam jawaban model.

### Etika AI
Dari segi etika, proyek ini menghadapi tantangan serius terkait akurasi dan keselamatan informasi, terlihat jelas dari halusinasi Model 2.0 yang memberikan instruksi menyesatkan (memasak rendang dikukus dan diberi keju) serta Model 1.0 yang mengalami pengulangan teks tak terkendali (spam). Kegagalan memberikan informasi yang benar ini berpotensi merugikan pengguna (pemborosan bahan makanan), sehingga secara etis saya wajib menyertakan disclaimer yang menyatakan bahwa AI masih dalam tahap eksperimen dan rentan terhadap bias pengetahuan umum yang menutupi pengetahuan spesifik lokal.

# Recommendation action

- Perkaya Dataset (Wajib): Tambah data latih jadi resep spesifik rendang untuk menghilangkan halusinasi (info ngawur).

- Tuning Parameter: Cegah pengulangan teks (spam) dengan menaikkan repetition_penalty ke 1.5 dan turunkan temperature ke 0.3.

- Safety UI: Tambahkan Disclaimer statis pada aplikasi bahwa resep yang dihasilkan AI memerlukan verifikasi manusia sebelum dimasak.

- Upgrade Model (Opsional): Jika spesifikasi memungkinkan, migrasi ke model yang lebih modern.